Michael Ricardo DS 2500 Project Trading torch

In [2]:
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
def fetch_ticker_data(ticker, years_ago= 5):
    """
    Gets the market data for a given date and ticker.
    Fetches from yfinance library.

    Args:
        years ago (int): Representing the most recent day for stock entries 
        ticker(str): Representing the offical company stock ticker
    Returns
        df (DataFrame): DataFrame with Stock pricing data and history data
    """
    most_recent = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    five_years_ago = (datetime.today() - timedelta(days= years_ago * 365)).strftime('%Y-%m-%d') 
    
    
    stock_data = yf.Ticker(ticker)
    returning_data = stock_data.history(start = five_years_ago, end= most_recent)
    
    return returning_data

In [ ]:
ticker = 'DIS'
dis_data = fetch_ticker_data(ticker)
dis_data = dis_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits'  ])

print(dis_data)

                                Close    Volume
Date                                           
2019-11-27 00:00:00-05:00  149.399445   6155400
2019-11-29 00:00:00-05:00  149.498077   6284900
2019-12-02 00:00:00-05:00  148.551254  10351000
2019-12-03 00:00:00-05:00  146.539291   9273800
2019-12-04 00:00:00-05:00  146.243393   7684800
...                               ...       ...
2024-11-18 00:00:00-05:00  113.540001  14236600
2024-11-19 00:00:00-05:00  112.419998  12166800
2024-11-20 00:00:00-05:00  114.260002   9319300
2024-11-21 00:00:00-05:00  114.720001  10261000
2024-11-22 00:00:00-05:00  115.650002  10093700

[1256 rows x 2 columns]


Adding variables not related to security itself: 

In [ ]:
#Data for the S&P 500 
ticker = '^GSPC'
SP500_data = fetch_ticker_data(ticker)
SP500_data = SP500_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits'  ])

print(SP500_data)

                                 Close      Volume
Date                                              
2019-11-27 00:00:00-05:00  3153.629883  3035470000
2019-11-29 00:00:00-05:00  3140.979980  1743420000
2019-12-02 00:00:00-05:00  3113.870117  3285750000
2019-12-03 00:00:00-05:00  3093.199951  3671580000
2019-12-04 00:00:00-05:00  3112.760010  3702980000
...                                ...         ...
2024-11-18 00:00:00-05:00  5893.620117  3983860000
2024-11-19 00:00:00-05:00  5916.979980  4036940000
2024-11-20 00:00:00-05:00  5917.109863  3772620000
2024-11-21 00:00:00-05:00  5948.709961  4230120000
2024-11-22 00:00:00-05:00  5969.339844  4141420000

[1256 rows x 2 columns]
